In [9]:
'''Generate Samples'''
import numpy as np
import pandas as pd
import random

# Define parameters for the 1D Ising Model
num_nodes = 10
num_edges = num_nodes - 1

# Define number of samples
num_samples = 100

# Function to solve the spin configuration
def spin_config_solver(edge_weights): # input: the "edge_weights" list
    spins = [random.choice([-1, 1])] # choose the first spin randomly from -1 and 1
    for i in range(num_edges):
        if edge_weights[i] >= 0:
            spins.append(spins[i]) # if the i-th edge-weight >= 0, then the next spin is set equal to the current spin
        else:
            spins.append(-spins[i]) # if the i-th edge-weith is less than 0, then the next spin is set equal to the opposite of the current spin
    return spins

# Function to calculate the Hamiltonian
def hamiltonian_solver(edge_weights, spins): # input: "edge_weights" list, "spins" list
    hamiltonian = 0
    for i in range(num_edges):
        hamiltonian += -edge_weights[i] * spins[i] * spins[i + 1]
    return hamiltonian

# Generate samples
samples = []
for _ in range(num_samples):
    # Randomly assign weights to the edges (between -1 and 1)
    edge_weights = np.random.uniform(-1, 1, num_edges)
    # Solve for spin configuration
    spins = spin_config_solver(edge_weights)
    # Calculate the Hamiltonian
    hamiltonian = hamiltonian_solver(edge_weights, spins)
    # Store the result
    samples.append({
        "edge_weights": edge_weights,
        "spins": spins,
        "hamiltonian": hamiltonian
    })

# Convert samples to a pandas DataFrame
samples_df = pd.DataFrame([{**{
    f"edge_{i}-{i+1}": weights[i] for i in range(len(weights))
}, **{
    f"spin_{i}": spins[i] for i in range(len(spins))
}, "hamiltonian": hamiltonian} for sample in samples for weights, spins, hamiltonian in [(sample["edge_weights"], sample["spins"], sample["hamiltonian"])]])

samples_df.head()


,edge_0-1,edge_1-2,edge_2-3,edge_3-4,edge_4-5,edge_5-6,edge_6-7,edge_7-8,edge_8-9,spin_0,spin_1,spin_2,spin_3,spin_4,spin_5,spin_6,spin_7,spin_8,spin_9,hamiltonian
0,0.534590,-0.757698,0.820187,0.164828,0.780173,-0.890411,-0.597613,0.055113,-0.868207,1,1,-1,-1,-1,-1,1,-1,-1,1,-5.468818
1,-0.364206,0.436819,0.580267,0.405104,-0.318310,0.568685,-0.196641,-0.068186,-0.674183,-1,1,1,1,1,-1,-1,1,-1,1,-3.612401
2,-0.106372,0.022995,-0.559961,0.497772,-0.959929,0.279753,0.144432,0.435240,0.626877,-1,1,1,-1,-1,1,1,1,1,1,-3.633331
3,0.655045,-0.225907,0.342875,-0.313429,0.883612,0.577112,-0.461958,0.184769,0.626901,1,1,-1,-1,1,1,1,-1,-1,-1,-4.271609
4,0.689794,0.791649,0.050532,0.735038,-0.535049,-0.898734,0.863661,0.847892,-0.719483,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-6.131832


In [2]:
# def calculate_opposite_absolute_sum(weights):
#     return -np.sum(np.abs(weights))

# samples_df["opposite_abs_sum_of_edge_weights"] = samples_df.apply(
#     lambda row: calculate_opposite_absolute_sum([row[f"edge_{i}-{i+1}"] for i in range(num_edges)]), axis=1
# )

# samples_df.head()

,edge_0-1,edge_1-2,edge_2-3,edge_3-4,edge_4-5,edge_5-6,edge_6-7,edge_7-8,edge_8-9,spin_0,...,spin_2,spin_3,spin_4,spin_5,spin_6,spin_7,spin_8,spin_9,hamiltonian,opposite_abs_sum_of_edge_weights
0,-0.850120,-0.101676,0.411878,-0.056664,0.325249,0.134893,-0.065505,0.866739,-0.424992,-1,...,-1,-1,1,1,1,-1,-1,1,-3.237717,-3.237717
1,-0.316317,0.767877,-0.080166,0.756458,0.524483,-0.874196,-0.182088,0.837825,0.742174,1,...,-1,1,1,1,-1,1,1,1,-5.081585,-5.081585
2,0.893560,-0.819725,0.305784,-0.759857,-0.287776,-0.260347,0.440156,-0.531989,-0.343900,1,...,-1,-1,1,-1,1,1,-1,1,-4.643093,-4.643093
3,0.331697,-0.534931,-0.879002,0.464513,-0.074494,-0.204365,0.559636,0.680599,0.665327,1,...,-1,1,1,-1,1,1,1,1,-4.394562,-4.394562
4,0.881537,0.183870,0.542704,0.090731,-0.547609,-0.283831,0.294172,0.190148,-0.840338,-1,...,-1,-1,-1,1,-1,-1,-1,1,-3.854941,-3.854941


In [10]:
# Save the DataFrame to a CSV file
samples_df.to_csv("1d_straight_ising_model_samples.csv", index=False)

Estimate the Hamiltonian using edge weights, by linear regression

In [44]:
data = pd.read_csv("1d_straight_ising_model_samples.csv")
data.head()

,edge_0-1,edge_1-2,edge_2-3,edge_3-4,edge_4-5,edge_5-6,edge_6-7,edge_7-8,edge_8-9,spin_0,spin_1,spin_2,spin_3,spin_4,spin_5,spin_6,spin_7,spin_8,spin_9,hamiltonian
0,0.534590,-0.757698,0.820187,0.164828,0.780173,-0.890411,-0.597613,0.055113,-0.868207,1,1,-1,-1,-1,-1,1,-1,-1,1,-5.468818
1,-0.364206,0.436819,0.580267,0.405104,-0.318310,0.568685,-0.196641,-0.068186,-0.674183,-1,1,1,1,1,-1,-1,1,-1,1,-3.612401
2,-0.106372,0.022995,-0.559961,0.497772,-0.959929,0.279753,0.144432,0.435240,0.626877,-1,1,1,-1,-1,1,1,1,1,1,-3.633331
3,0.655045,-0.225907,0.342875,-0.313429,0.883612,0.577112,-0.461958,0.184769,0.626901,1,1,-1,-1,1,1,1,-1,-1,-1,-4.271609
4,0.689794,0.791649,0.050532,0.735038,-0.535049,-0.898734,0.863661,0.847892,-0.719483,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-6.131832


In [45]:
X = data.iloc[:,0:9]

print(type(X))
print(X.iloc[0:5])
X.head()

<class 'pandas.core.frame.DataFrame'>
   edge_0-1  edge_1-2  edge_2-3  edge_3-4  edge_4-5  edge_5-6  edge_6-7  \
0  0.534590 -0.757698  0.820187  0.164828  0.780173 -0.890411 -0.597613   
1 -0.364206  0.436819  0.580267  0.405104 -0.318310  0.568685 -0.196641   
2 -0.106372  0.022995 -0.559961  0.497772 -0.959929  0.279753  0.144432   
3  0.655045 -0.225907  0.342875 -0.313429  0.883612  0.577112 -0.461958   
4  0.689794  0.791649  0.050532  0.735038 -0.535049 -0.898734  0.863661   

   edge_7-8  edge_8-9  
0  0.055113 -0.868207  
1 -0.068186 -0.674183  
2  0.435240  0.626877  
3  0.184769  0.626901  
4  0.847892 -0.719483  


,edge_0-1,edge_1-2,edge_2-3,edge_3-4,edge_4-5,edge_5-6,edge_6-7,edge_7-8,edge_8-9
0,0.534590,-0.757698,0.820187,0.164828,0.780173,-0.890411,-0.597613,0.055113,-0.868207
1,-0.364206,0.436819,0.580267,0.405104,-0.318310,0.568685,-0.196641,-0.068186,-0.674183
2,-0.106372,0.022995,-0.559961,0.497772,-0.959929,0.279753,0.144432,0.435240,0.626877
3,0.655045,-0.225907,0.342875,-0.313429,0.883612,0.577112,-0.461958,0.184769,0.626901
4,0.689794,0.791649,0.050532,0.735038,-0.535049,-0.898734,0.863661,0.847892,-0.719483


In [75]:
Y = data['hamiltonian']
Y.head()

0   -5.468818
1   -3.612401
2   -3.633331
3   -4.271609
4   -6.131832
Name: hamiltonian, dtype: float64

In [52]:
def featurization(X):
    X_feat = X.copy()
    X_feat = abs(X_feat)
    X_feat.insert(0, 'x0', np.ones(len(X_feat.index))) # X_feat.index represents the row labels, len(X_feat.index) gives the number of rows in the DataFrame
    return X_feat

X_feat = featurization(X)

print(X_feat.index)

print(X_feat)
print(X_feat.shape)
X_feat.head()

RangeIndex(start=0, stop=100, step=1)
     x0  edge_0-1  edge_1-2  edge_2-3  edge_3-4  edge_4-5  edge_5-6  edge_6-7  \
0   1.0  0.534590  0.757698  0.820187  0.164828  0.780173  0.890411  0.597613   
1   1.0  0.364206  0.436819  0.580267  0.405104  0.318310  0.568685  0.196641   
2   1.0  0.106372  0.022995  0.559961  0.497772  0.959929  0.279753  0.144432   
3   1.0  0.655045  0.225907  0.342875  0.313429  0.883612  0.577112  0.461958   
4   1.0  0.689794  0.791649  0.050532  0.735038  0.535049  0.898734  0.863661   
..  ...       ...       ...       ...       ...       ...       ...       ...   
95  1.0  0.154289  0.139238  0.583644  0.488128  0.414636  0.293932  0.178772   
96  1.0  0.763359  0.857441  0.158695  0.172001  0.303062  0.858090  0.829362   
97  1.0  0.987690  0.270369  0.351431  0.576336  0.053048  0.234050  0.044328   
98  1.0  0.460589  0.061897  0.644780  0.232530  0.685908  0.211070  0.765385   
99  1.0  0.095459  0.878381  0.047448  0.730944  0.481841  0.356092  0.

,x0,edge_0-1,edge_1-2,edge_2-3,edge_3-4,edge_4-5,edge_5-6,edge_6-7,edge_7-8,edge_8-9
0,1.0,0.534590,0.757698,0.820187,0.164828,0.780173,0.890411,0.597613,0.055113,0.868207
1,1.0,0.364206,0.436819,0.580267,0.405104,0.318310,0.568685,0.196641,0.068186,0.674183
2,1.0,0.106372,0.022995,0.559961,0.497772,0.959929,0.279753,0.144432,0.435240,0.626877
3,1.0,0.655045,0.225907,0.342875,0.313429,0.883612,0.577112,0.461958,0.184769,0.626901
4,1.0,0.689794,0.791649,0.050532,0.735038,0.535049,0.898734,0.863661,0.847892,0.719483


In [77]:
# Define Cost Function
def cost(X_feat, y, theta):
    ''' Function to calculate cost function assuming a hypothesis of form h = theta.T*X
  Inputs:
  X_feat = feature matrix
  y = ground true of training data
  theta = array of parameters for hypothesis

  Returns:
  J = cost function
  '''
    m = len(y)
    h = np.dot(theta, X_feat.T) # hypothesis
    J = (1/(2*m)) * np.sum( (h - y)**2 )
    return J

def jacobian(X_feat, y, theta):
    ''' Function to calculate jacobian assuming a hypothesis of form h = theta.T*X
    Inputs:
    X_feat = feature matrix
    y = ground true of training data
    theta = array of parameters for hypothesis

    Outputs:
    Jac = Jacobian of the hypothesis h = theta.T*X
    '''
    m = len(y)
    h = np.dot(theta, X_feat.T) # hypothesis
    Jac = (1/m) * np.dot((h-y), X_feat)
    return Jac


# Define Gradient Descent Algorithm
def gradient_descent(X_feat, y, theta, lr, num_iters):
    '''Gradient descent algorithm
    Inputs:
    X_feat = features
    y = training data ground true
    theta = parameters of the hypothesis
    lr = learning rate
    num_iters = number of iterations

    Output:
    theta = final parameters
    J_hist = array of cost as a function of iterations
    '''
    m = len(y)
    J_hist = np.zeros(num_iters)





In [80]:
import random

theta = np.random.uniform(-2, 2, X_feat.shape[1])

h = np.dot(theta, X_feat.T)

Jac = jacobian(X_feat, Y, theta)
print(Jac.shape)

(10,)
